# Baixar e Instalar pacotes de bibliotecas necessárias

In [ ]:
!pip install langchain_huggingface
!pip install langchain_community
!pip install pypdf
!pip install sentence-transformers

# Sugerir carregar Google Drive para os documentos a serem carregados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Carregar documentos e adicionar em uma lista de documentos

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
import os

# Lista para armazenar os documentos carregados
diretorio_base='/content/drive/MyDrive/topicos_ia/data/minuta'
docs_list = []
i = 1

# Itera sobre os arquivos no diretório e carrega os PDFs
for nome_arquivo in os.listdir(diretorio_base):
    if nome_arquivo.endswith(".pdf"):  # Verifica se o arquivo é um PDF
        caminho_pdf = os.path.join(diretorio_base, nome_arquivo)

        loader = PyPDFLoader(caminho_pdf)

        print(f'loader: {loader}\n\n')
                
        docs = await loader.aload()

        print(f'docs_loader: {docs[0].page_content[:100]}\n\n')
        print(docs[0].metadata)
        
        docs_list.append(docs)

        print(i)
        i += 1

    if i == 3:
      break


print(docs_list)
print(len(docs_list))

# Dividir os Documentos em partes menores e adicionar elas em uma lista especifica de cada tamanho

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1ª variação
text_splitter_1 = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300
)

# 2ª variação
text_splitter_2 = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=600
)

# 3ª variação
text_splitter_3 = RecursiveCharacterTextSplitter(
    chunk_size=2500,
    chunk_overlap=900
)

chunks1_list = []
chunks2_list = []
chunks3_list = []

for i, doc in enumerate(docs_list):
  # print(doc[i])
  print(doc[i].metadata)
  chunk_1 = text_splitter_1.split_text(doc[i].page_content)
  chunk_2 = text_splitter_2.split_text(doc[i].page_content)
  chunk_3 = text_splitter_3.split_text(doc[i].page_content)

  chunks1_list.append(chunk_1)
  chunks2_list.append(chunk_2)
  chunks3_list.append(chunk_3)


print('chunks 1')
for i in chunks1_list:
  print(i, end='\n')

print('\nchunks 2')
for i in chunks2_list:
  print(i, end='\n')

print('\nchunks 3')
for i in chunks3_list:
  print(i)

# Dicionario de Embeddings

In [ ]:
embedding_models = {
    'all-MiniLM-L6-v2':
        'sentence-transformers/all-MiniLM-L6-v2',
        
    'all-mpnet-base-v2':
        'sentence-transformers/all-mpnet-base-v2',
        
    'multi-qa-MiniLM-L6-cos-v1':
        'sentence-transformers/multi-qa-MiniLM-L6-cos-v1',
}

# Instanciar os Embeddings e adicionar em uma lista

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_list = []

embedding1 = HuggingFaceEmbeddings(model_name=embedding_models['all-MiniLM-L6-v2'])
embedding2 = HuggingFaceEmbeddings(model_name=embedding_models['all-mpnet-base-v2'])
embedding3 = HuggingFaceEmbeddings(model_name=embedding_models['multi-qa-MiniLM-L6-cos-v1'])

embedding_list.append(embedding1)
embedding_list.append(embedding2)
embedding_list.append(embedding3)

# Baixar e instalar os pacotes para o FAISS

In [ ]:
!pip install faiss-cpu
!pip install faiss-gpu
!pip install tiktoken

## Converter novamente os documentos divididos (strings) para objetos Document e criar índices e Embeddings deles, por fim adicionar em uma lista com os embeddings de tamanho especificos que foram criados com base na divisão dos chunks/overlaps

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.schema import Document

# Certificar que cada chunk seja uma string (caso seja uma lista de palavras ou sentenças)
chunks1_docs = [Document(page_content=" ".join(chunk) if isinstance(chunk, list) else chunk) for chunk in chunks1_list]
chunks2_docs = [Document(page_content=" ".join(chunk) if isinstance(chunk, list) else chunk) for chunk in chunks2_list]
chunks3_docs = [Document(page_content=" ".join(chunk) if isinstance(chunk, list) else chunk) for chunk in chunks3_list]


# Criar os índices FAISS
faiss1_list = []
faiss2_list = []
faiss3_list = []

for i, embedding in enumerate(embedding_list):
    print(f"Gerando índice FAISS com o modelo: {embedding.model_name}")

    faiss_index1 = FAISS.from_documents(chunks1_docs, embedding)
    faiss_index2 = FAISS.from_documents(chunks2_docs, embedding)
    faiss_index3 = FAISS.from_documents(chunks3_docs, embedding)

    # Adicionando os índices a uma lista
    faiss1_list.append(faiss_index1)
    faiss2_list.append(faiss_index2)
    faiss3_list.append(faiss_index3)

    # Salvar o índice FAISS localmente
    faiss_index1.save_local("faiss_index1.index")
    faiss_index2.save_local("faiss_index2.index")
    faiss_index3.save_local("faiss_index3.index")

    print("Índices salvos com sucesso!")

# Carregar os índices FAISS a partir dos arquivos salvos
faiss_index1_loaded = FAISS.load_local("faiss_index1.index", embedding1, allow_dangerous_deserialization=True)
faiss_index2_loaded = FAISS.load_local("faiss_index2.index", embedding2, allow_dangerous_deserialization=True)
faiss_index3_loaded = FAISS.load_local("faiss_index3.index", embedding3, allow_dangerous_deserialization=True)

print("Índices carregados com sucesso!")

# Testar se a similaridade dos vetores de embeddings são carregadas

In [ ]:
# Acessando os vetores de embeddings a partir do índice FAISS
faiss_vectors1 = faiss_index1_loaded.index.reconstruct_n(0, faiss_index1_loaded.index.ntotal)
faiss_vectors2 = faiss_index2_loaded.index.reconstruct_n(0, faiss_index2_loaded.index.ntotal)
faiss_vectors3 = faiss_index3_loaded.index.reconstruct_n(0, faiss_index3_loaded.index.ntotal)

# Visualizando os vetores
print("Vetores do índice 1:")
print(faiss_vectors1)

print("Vetores do índice 2:")
print(faiss_vectors2)

print("Vetores do índice 3:")
print(faiss_vectors3)